In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import pickle
from tensorflow.keras.models import Model
from tensorflow.keras.layers import *
from keras.models import load_model

In [4]:
embedding_matrix=pickle.load(open('embed_mat1.pkl','rb'))
tokenizer=pickle.load(open('tokenizer1.pkl','rb'))

In [5]:
vocab_size=(len(tokenizer.word_index)+1)

In [7]:
max_len=39
max_len_dec=44

### Model

In [9]:
path='/content/gdrive/MyDrive/CS_2/CS2Model'

In [10]:
model = tf.saved_model.load(path)

### Predictions

In [11]:
def predict(input_sentence, model):

  input_sequence=tokenizer.texts_to_sequences([input_sentence])
  inputs=pad_sequences(input_sequence, maxlen=max_len, padding='post')
  inputs=tf.convert_to_tensor(inputs)
  result=''
  units=128
  hidden=[tf.zeros((1,units))]
  encoder_output,hidden_state,cell_state=model.encoder(inputs)
  dec_hidden=hidden_state
  dec_input=tf.expand_dims([tokenizer.word_index['<start>']],0)
  for t in range(40):
      predictions,dec_hidden,cell_state,attention_weights,context_vector=model.decoder.onestepdecoder((dec_input,encoder_output,dec_hidden,cell_state))

      predicted_id=tf.argmax(predictions[0]).numpy()
      result+=tokenizer.index_word[predicted_id]+' '
      if tokenizer.word_index['<end>']==predicted_id:
          return result
      dec_input= tf.expand_dims([predicted_id],0)
  return result

In [15]:
data=pd.read_csv('data_wl')

In [ ]:
for i in range(0,10):
    input_sentence=data.Source.iloc[i]
    print('Input:',input_sentence)
    print('Prediction:',predict(input_sentence, model))
    print('Actual:',data.Target_out.iloc[i])
    print('---------------------------------------')

Input: Violyn can you intro.?
Prediction: Violyn can you introduce <end> 
Actual: Violyn, can you introduce? <end>
---------------------------------------
Input: Yea...Its a superstition thing. Go temple n take e first joss stick. Its always featured in news lei... Last min hug buddha leg. Not good... :p
Prediction: Yea it's superstition Have to the temple to the weirdest track be a bit of the news Doing last minute reply But my dear friend got no school <end> 
Actual: Yea, it's superstition. Have to go to the temple to take the first joss stick. It's always featured in the news. Doing last minute work, it's not good. <end>
---------------------------------------
Input: Im not workg...Wat time junmei aft work? I saw david ytd...
Prediction: I'm not not not sure I'll not be late I have photograph <end> 
Actual: I'm not working. What time is Junmei after work? I saw David yesterday. <end>
---------------------------------------
Input: Can u pls call me?
Prediction: Can you call me me <en